In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot

In [2]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Union\files\union01.pdf'
tables=tabula.read_pdf(f,
                          lattice=True,
                          pages="all",
                          silent=True,
                          multiple_tables=True,
                          pandas_options={'header':None})
df = pd.DataFrame()
df = pd.concat([c for c in tables]).drop_duplicates()

In [4]:
df.head()

,0,1,2,3,4
0,Tran Id,Tran Date,Remarks,Amount (Rs.),Balance (Rs.)
1,S55979126,28/09/2019,MOBFT to: SABDUL\rSK/927116596088,100.00 (Dr),3052.59
2,S53394988,27/09/2019,IMPSAB/927018826238/UBIN0550\r990/9811837131,1645.00 (Cr),3152.59
3,S51939598,27/09/2019,ASHOK KUMAR ROUT,31200.00 (Dr),1507.59
4,AA41707,27/09/2019,BY CASH,7000.00 (Cr),32707.59


In [5]:
df=df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)
try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('description').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('remark').any(),axis=1)==True].index][0]
            df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
        except:
            print("\nUnion Column Headers Missing"); pass
try:
    df = df.drop(["Init."], axis=1)
except:
    try:
        df = df.drop(["Branch Name"], axis=1)
    except:
        try:
            df = df.drop(["Tran Id"], axis=1)
        except:pass

In [6]:
df.head()

,Tran Date,Remarks,Amount (Rs.),Balance (Rs.)
0,28/09/2019,MOBFT to: SABDUL\rSK/927116596088,100.00 (Dr),3052.59
1,27/09/2019,IMPSAB/927018826238/UBIN0550\r990/9811837131,1645.00 (Cr),3152.59
2,27/09/2019,ASHOK KUMAR ROUT,31200.00 (Dr),1507.59
3,27/09/2019,BY CASH,7000.00 (Cr),32707.59
4,26/09/2019,NEFT:USHA TECH\rCONSTRUCTION\rMAHBH19269280247,18900.00 (Cr),25707.59


In [7]:
df.tail()

,Tran Date,Remarks,Amount (Rs.),Balance (Rs.)
261,13/06/2019,4520555099035913/916411033732/\r509901010240919,1200.00 (Dr),1975.00
262,13/06/2019,4520555099035913/916411030480/\r509901010240919,2500.00 (Dr),3175.00
263,12/06/2019,MOBMOBILE\rRECHARGE/916308693420,199.00 (Dr),5675.00
264,11/06/2019,POS:DAINIK\rBAZAAR/GURGAON/916212835\r962,2372.00 (Dr),5874.00
265,11/06/2019,NEFT:USHATECH PROJECTS\rPRIVATE LIMITED\rMAHBH...,7600.00 (Cr),8246.00


In [8]:
df = df[~df.index.isin(df[df.apply(lambda row:row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(),axis=1)==True].index)]
#df = pd.DataFrame(df.T.drop_duplicates().T)
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1, inplace=True)

In [9]:
df.head()

,Tran Date,Remarks,Amount (Rs.),Balance (Rs.)
0,28/09/2019,MOBFT to: SABDUL\rSK/927116596088,100.00 (Dr),3052.59
1,27/09/2019,IMPSAB/927018826238/UBIN0550\r990/9811837131,1645.00 (Cr),3152.59
2,27/09/2019,ASHOK KUMAR ROUT,31200.00 (Dr),1507.59
3,27/09/2019,BY CASH,7000.00 (Cr),32707.59
4,26/09/2019,NEFT:USHA TECH\rCONSTRUCTION\rMAHBH19269280247,18900.00 (Cr),25707.59


In [ ]:
try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
except: print("\nBalance columns missing") 

try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
    except: pass 

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except:
    try:
        dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
    except: pass

df[[wdl,dep]] = df[[wdl,dep]].replace({"-":np.nan, "NA":np.nan})

df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace(",","").replace("Cr","").replace("Dr","")).astype(float)
#print(df.head())

df.to_csv("parsed.csv",index=0)
print(df)